In [21]:
import pandas as pd
import shutil
from datetime import datetime

In [22]:
drz_eaf_csv='../data/tira-drz/eaf_data.csv'
df = pd.read_csv(drz_eaf_csv)
df.head()

,start,end,text,tier,wav_source,eaf_name
0,44650.0,47200.0,kúkù kàpɔ́ ŋɔ́mɔ̀,IPA Transcription,/G:/Shared drives/Tira/Recordings/2022-10-21-T...,/Users/markjos/Google Drive/My Drive/Speechpro...
1,49505.0,51325.0,kúkù kàpɔ́ ŋɔ́mɔ̀,IPA Transcription,/G:/Shared drives/Tira/Recordings/2022-10-21-T...,/Users/markjos/Google Drive/My Drive/Speechpro...
2,82735.0,84685.0,kúkù kápɛ̀ ŋɔ̀mɔ̀,IPA Transcription,/G:/Shared drives/Tira/Recordings/2022-10-21-T...,/Users/markjos/Google Drive/My Drive/Speechpro...
3,87150.0,88890.0,kúkù kápɛ̀ ŋɔ̀mɔ̀,IPA Transcription,/G:/Shared drives/Tira/Recordings/2022-10-21-T...,/Users/markjos/Google Drive/My Drive/Speechpro...
4,107860.0,109420.0,kúkù kâpɔ́ ŋɔ́mɔ̀,IPA Transcription,/G:/Shared drives/Tira/Recordings/2022-10-21-T...,/Users/markjos/Google Drive/My Drive/Speechpro...


In [16]:
df['tier'].unique()

array(['IPA Transcription', 'Word', 'Translation', 'Annotation (other)',
       'Gloss', 'Tira orthography', 'HIM', 'SHA', 'PET', 'NIN', 'MAR',
       'MISC'], dtype=object)

In [18]:
print(df.shape)
unneeded_tiers = ['Word', 'Annotation (other)', 'Gloss', 'Tira orthography']
df = df[~df['tier'].isin(unneeded_tiers)]
df.shape

(40304, 6)


(38828, 6)

In [20]:
# find intervals where a diarization label has an existing transcription and translation
df_wide = df.copy()
df_wide['translation']=''
df_wide['human_transcription']=''
# slice all rows containing a transcription or translation interval
text_intervals = df['tier'].isin(['IPA Transcription', 'Translation'])
for i, row in df[text_intervals].iterrows():
    tier = row['tier']
    eaf_name = row['eaf_name']
    start = row['start']
    text = row['text']

    aligned_start = (df_wide['eaf_name']==eaf_name) & (df_wide['start']==start) & (df['tier']=='HIM')
    if df_wide[aligned_start].empty:
        continue
    assert len(df[aligned_start])==1, df[aligned_start]

    if tier=='IPA transcription':
        df_wide.loc[aligned_start,'human_transcription']=text
    else:
        df_wide.loc[aligned_start,'translation']=text
df_wide = df_wide[~text_intervals]
print(df_wide.shape)
df_wide.head()

(16931, 8)


,start,end,text,tier,wav_source,eaf_name,translation,human_transcription
258,46273.0,48215.0,TIC,HIM,/Volumes/GoogleDrive/Shared drives/Tira/Record...,/Users/markjos/Google Drive/My Drive/Speechpro...,(you can say) ŋɛ́n ŋɔ̂ɽàt̪à nɛ̀ lə̀vɛ̌r,
259,53609.0,55170.0,TIC,HIM,/Volumes/GoogleDrive/Shared drives/Tira/Record...,/Users/markjos/Google Drive/My Drive/Speechpro...,(yeah) ŋɛ́n ŋɔ̂ɽàt̪à nɛ̀ lə̀vɛ̌r,
260,89735.0,91750.0,TIC,HIM,/Volumes/GoogleDrive/Shared drives/Tira/Record...,/Users/markjos/Google Drive/My Drive/Speechpro...,ŋɛ́n ŋɔ̂ɽàt̪ɔ́ nɛ̀ lə̀vɛ̌r,
261,101726.0,104826.0,TIC,HIM,/Volumes/GoogleDrive/Shared drives/Tira/Record...,/Users/markjos/Google Drive/My Drive/Speechpro...,ŋɛ́n ŋɔ̂ɽàt̪ɔ́ nɛ̀ lə̀vɛ̀r,
262,114226.0,116309.0,TIC,HIM,/Volumes/GoogleDrive/Shared drives/Tira/Record...,/Users/markjos/Google Drive/My Drive/Speechpro...,ŋɛ́n ŋɔ̂ɽàt̪ɔ́ nɛ̀ lə̀vɛ̀r,


In [26]:
now = datetime.now()
now_fp_friendly = str(now).replace(' ', '-')
now_fp_friendly = str(now_fp_friendly).replace(':', '-')
now_fp_friendly = str(now_fp_friendly).replace('.', '-')
backup_path=drz_eaf_csv.replace('.csv', now_fp_friendly+'.csv')
shutil.move(drz_eaf_csv, backup_path)
df_wide.to_csv(drz_eaf_csv, index=False)

'2024-07-25-17-16-30-474521'